<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/Kcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/K-project/output/'
data_path = '/content/drive/MyDrive/K-project/'
# pickle 경로를 설정하기 위해

In [ ]:
# 한국시간 설정
import pytz

KST = pytz.timezone('Asia/Seoul')
today = str(datetime.now(KST).strftime("%Y%m%d"))
print(today)

In [ ]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
fullcode

In [ ]:
#상장폐지 상태 제외시키기.
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']
fullcode_filter

In [ ]:
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
# fullcode열을 list로 변경 후 확인

In [ ]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

strtDd = '19000101'
endDd = today

for i in fullcode_list[:2]: # 테스트를 위해 2개만 실행 (두 종목만 출력됨)
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': i,
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2=pd.DataFrame(j2['output'])
    # print(df2)
    # print('------------------------------------------------')

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': i,
    'isuCd2': i,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3=pd.DataFrame(j2['output'])
    # print(df3)
    # print('------------------------------------------------')

    df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    df3_ = df3[['TRD_DD', 'EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]

    result = pd.merge(df2_, df3_)
    print(result)
    print('------------------------------------------------')

In [ ]:
import talib as ta

In [ ]:
result

In [ ]:
result['TDD_CLSPRC'] = result['TDD_CLSPRC'].str.replace(',','').astype('float')
result['TDD_OPNPRC'] = result['TDD_OPNPRC'].str.replace(',','').astype('float')
result['TDD_HGPRC'] = result['TDD_HGPRC'].str.replace(',','').astype('float')
result['TDD_LWPRC'] = result['TDD_LWPRC'].str.replace(',','').astype('float')

result.dtypes

In [ ]:
#1-GDC
import pandas as pd
import talib as ta
import numpy as np

#단순 이동평균-단기이평선(20일), 장기이평선(200일)
short_ma = ta._ta_lib.MA('TDD_CLSPRC', timeperiod=20, matype=0)
long_ma = ta._ta_lib.MA('TDD_CLSPRC', timeperiod=200, matype=0)

In [ ]:
#2-RSI
import talib as ta
result['RSI'] = ta._ta_lib.RSI(result['TDD_CLSPRC'],timeperiod=14)
result['RSI'] = result['RSI'].fillna(method='bfill')
result

In [ ]:
import matplotlib.pyplot as plt
result['RSI']=ta.RSI(result['RSI'],14)
result['RSI'].plot(figsize=(12,10))
plt.show()

In [ ]:
#3-ROC
result['ROC'] = ta._ta_lib.ROC(result['TDD_CLSPRC'],timeperiod=10)
result['ROC'] = result['ROC'].fillna(method='bfill')
result

In [ ]:
result['ROC']=ta.ROC(result['ROC'],14)
result['ROC'].plot(figsize=(12,10))
plt.show()

In [ ]:
#4-STC
import talib.abstract as ta
result['STOCH'] = ta._ta_lib.STOCH('TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC',fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

In [ ]:
#보기쉬운 행으로 바꾸기
result.columns = ['거래일자', '종목코드','종목명','종가','시가','고가','저가',
                  '시가총액','EPS','PER','BPS','PBR','주당배당금','배당수익률']
result